# train

In [1]:
import yaml
import os

with open("./config.yaml", 'rb') as f:
    config = yaml.load(f)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  """


In [2]:
OUTPUT_ROOT = config['IO_OPTION']['OUTPUT_ROOT']
os.makedirs(OUTPUT_ROOT, exist_ok=True)

## load library

In [3]:
# python default library
import os
import shutil
import datetime
import sys
import pickle

# general analysis tool-kit
import numpy as np
import pandas as pd
#from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns

# pytorch
import torch
from torch import optim, nn
from torch.utils.tensorboard import SummaryWriter
import timm

#from torchvision.models import wide_resnet50_2, resnet18
#from torchsummary import summary

# etc
import yaml
yaml.warnings({'YAMLLoadWarning': False})

# original library
import common as com
import pytorch_modeler as modeler
from pytorch_model import EfficientNet_b1 as Model
#from pytorch_utils import filtered_load_model
#import models

import librosa
import IPython
import librosa.display

In [4]:
# effnet1 = timm.create_model('efficientnet_b1', pretrained=True)
# # M7:block[5], M8:block[6], M9:conv_head
# effnet1

## load config and set logger

In [5]:
log_folder = config['IO_OPTION']['OUTPUT_ROOT']+'/{0}.log'.format(datetime.date.today())
logger = com.setup_logger(log_folder, '00_train.py')

## Setting

In [6]:
# Setting seed
modeler.set_seed(42)

In [7]:
############################################################################
# Setting I/O path
############################################################################
# input dirs
INPUT_ROOT = config['IO_OPTION']['INPUT_ROOT']
dev_dir = INPUT_ROOT + "/dev_data"
add_dev_dir = INPUT_ROOT + "/add_dev_data"
# machine type
machine_types = os.listdir(dev_dir)
# output dirs
OUTPUT_ROOT = config['IO_OPTION']['OUTPUT_ROOT']
MODEL_DIR = config['IO_OPTION']['OUTPUT_ROOT'] + '/models'

TB_DIR = config['IO_OPTION']['OUTPUT_ROOT'] + '/tb'
OUT_FEATURE_DIR = OUTPUT_ROOT + '/extraction_features'
PRED_DIR = config['IO_OPTION']['OUTPUT_ROOT'] + '/pred'
SCORE_DIR = config['IO_OPTION']['OUTPUT_ROOT'] + '/score'
#os.makedirs(OUTPUT_ROOT, exist_ok=True)
os.makedirs(MODEL_DIR, exist_ok=True)
os.makedirs(TB_DIR, exist_ok=True)
os.makedirs(OUT_FEATURE_DIR, exist_ok=True)
os.makedirs(PRED_DIR, exist_ok=True)
os.makedirs(SCORE_DIR, exist_ok=True)

# copy config
# shutil.copy('./config.yaml', OUTPUT_ROOT)

In [8]:
torch.cuda.is_available()

True

## make path list

- __train_paths__ ([dict]): Each Machine Type
    - __ToyCar__ ([dict]): Each dataset
        - train ([list]) : paths
        - valid_source ([list]) : paths
        - valid_target ([list]) :paths
    - __ToyTrain__ ([dict]): Each dataset
        - train ([list]) : paths
        - valid_source ([list]) : paths
        - valid_target ([list]) : paths
    - ...

In [9]:
percent = 0.1

In [10]:
dev_paths = {}

for machine_type in machine_types:
    # dev train
    dev_train_paths = [f"{dev_dir}/{machine_type}/train/" + file for file in os.listdir(f"{dev_dir}/{machine_type}/train")]
    dev_train_paths = sorted(dev_train_paths)
    
    print(f'Machine {machine_type}', end=',')
    print(f'dev_train_size : {len(dev_train_paths)}', end=', ')
    dev_train_source_paths = [path for path in dev_train_paths if 'source' in path]
    print(f'source_size : {len(dev_train_source_paths)}', end=', ')
    dev_train_target_paths = [path for path in dev_train_paths if 'target' in path]
    print(f'target_size : {len(dev_train_target_paths)}', end=', ')
    # increse tgt
    num_increse_tgt = int(len(dev_train_source_paths) * percent)
    increse_dev_train_target_paths = dev_train_target_paths * num_increse_tgt
    # concat
    dev_train_paths = dev_train_source_paths + increse_dev_train_target_paths
    
    # add_dev train
    add_dev_paths = [f"{add_dev_dir}/{machine_type}/train/" + file for file in os.listdir(f"{add_dev_dir}/{machine_type}/train")]
    add_dev_paths = sorted(add_dev_paths)
    print(f'adddev_train_size : {len(add_dev_paths)}', end=', ')
    add_dev_source_paths = [path for path in add_dev_paths if 'source' in path]
    print(f'source_size : {len(add_dev_source_paths)}', end=', ')
    add_dev_target_paths = [path for path in add_dev_paths if 'target' in path]
    print(f'target_size : {len(add_dev_target_paths)}')
    # increse tgt
    num_increse_tgt = int(len(add_dev_source_paths) * percent)
    increse_add_dev_target_paths = add_dev_target_paths * num_increse_tgt
    # concat
    add_dev_paths = add_dev_source_paths + increse_add_dev_target_paths

    print('==============================================================')
    
    # dev_source valid
    dev_source_paths = [f"{dev_dir}/{machine_type}/source_test/" + file for file in os.listdir(f"{dev_dir}/{machine_type}/source_test")]
    dev_source_paths = sorted(dev_source_paths)
    
    # dev_target valid
    dev_target_paths = [f"{dev_dir}/{machine_type}/target_test/" + file for file in os.listdir(f"{dev_dir}/{machine_type}/target_test")]
    dev_target_paths = sorted(dev_target_paths)
    
    # bundle
    dev_paths[machine_type] = {}
    dev_paths[machine_type]['train'] = dev_train_paths + add_dev_paths
    dev_paths[machine_type]['valid_source'] = dev_source_paths
    dev_paths[machine_type]['valid_target'] = dev_target_paths

Machine pump,dev_train_size : 3009, source_size : 3000, target_size : 9, adddev_train_size : 3009, source_size : 3000, target_size : 9
Machine gearbox,dev_train_size : 3026, source_size : 3017, target_size : 9, adddev_train_size : 3114, source_size : 3105, target_size : 9
Machine slider,dev_train_size : 3009, source_size : 3000, target_size : 9, adddev_train_size : 3009, source_size : 3000, target_size : 9
Machine ToyCar,dev_train_size : 3009, source_size : 3000, target_size : 9, adddev_train_size : 3009, source_size : 3000, target_size : 9
Machine valve,dev_train_size : 3009, source_size : 3000, target_size : 9, adddev_train_size : 3009, source_size : 3000, target_size : 9
Machine ToyTrain,dev_train_size : 3009, source_size : 3000, target_size : 9, adddev_train_size : 3009, source_size : 3000, target_size : 9
Machine fan,dev_train_size : 3009, source_size : 3000, target_size : 9, adddev_train_size : 3009, source_size : 3000, target_size : 9


In [11]:
machine_types

['pump', 'gearbox', 'slider', 'ToyCar', 'valve', 'ToyTrain', 'fan']

In [12]:
#############################################################################
# run
#############################################################################
def run(machine_type, dev_paths):
    com.tic()
    os.makedirs(TB_DIR+'/'+machine_type, exist_ok=True)
    tb_log_dir = TB_DIR + '/' + machine_type
    writer = SummaryWriter(log_dir = tb_log_dir)
    
    logger.info('TARGET MACHINE_TYPE: {0}'.format(machine_type))
    logger.info('MAKE DATA_LOADER')
    # dev_train_paths
    dataloaders_dict = modeler.make_dataloader(dev_paths, machine_type)
    # parameter setting
    logger.info('TRAINING')
    model = Model(n_out=36, n_centers=6)
    
    optimizer = torch.optim.Adam(model.parameters(),lr=1e-4)
    output_dict, model, pred_df, score_df = modeler.run_training(model, dataloaders_dict, writer, optimizer)
    
    # output
    feature_out_path = f'{OUT_FEATURE_DIR}/{machine_type}_features.pkl'
    model_out_path = f'{MODEL_DIR}/{machine_type}_model.pkl'
    pred_out_path = f'{PRED_DIR}/{machine_type}_pred.csv'
    score_out_path = f'{SCORE_DIR}/{machine_type}_pred.csv'
    # save
    pd.to_pickle(output_dict, feature_out_path)
    torch.save(model.state_dict(), model_out_path)
    pred_df.to_csv(pred_out_path)
    score_df.to_csv(score_out_path)
    logger.info(f'SAVE SUCCESS : {model_out_path}')
    
    com.toc()

In [13]:
#run(machine_types[0], dev_paths)

In [ ]:
for machine_type in machine_types:
    run(machine_type, dev_paths)

2021-10-27 15:22:54,614 - 00_train.py - INFO - TARGET MACHINE_TYPE: pump
2021-10-27 15:22:54,614 - 00_train.py - INFO - MAKE DATA_LOADER
2021-10-27 15:22:54,620 - 00_train.py - INFO - TRAINING


use: cuda:0


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 600/600 [00:58<00:00, 10.34it/s]
2021-10-27 15:25:42,281 - pytorch_modeler.py - INFO - epoch:1/100, tr_loss:4.682612, src_loss:0.656038, src_mean_auc:0.643933, tgt_loss:0.564154, tgt_mean_auc:0.607433, mean_auc:0.625683,


,AUC,pAUC
Source_0,0.650800,0.594211
Source_1,0.672000,0.563158
Source_2,0.609000,0.510000
Target_0,0.660500,0.547368
Target_1,0.583800,0.482105
Target_2,0.578000,0.520526
mean,0.625683,0.536228
h_mean,0.623459,0.533728


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 45/45 [00:48<00:00,  1.08s/it]
2021-10-27 15:26:31,030 - pytorch_modeler.py - INFO - epoch:2/100, tr_loss:2.240242, src_loss:0.656038, src_mean_auc:0.643933, tgt_loss:0.564154, tgt_mean_auc:0.607433, mean_auc:0.625683,


...

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 45/45 [00:47<00:00,  1.06s/it]
2021-10-27 16:52:16,333 - pytorch_modeler.py - INFO - epoch:89/100, tr_loss:0.035380, src_loss:0.273854, src_mean_auc:0.764667, tgt_loss:0.304583, tgt_mean_auc:0.719867, mean_auc:0.742267,


,AUC,pAUC
Source_0,0.797500,0.615263
Source_1,0.828800,0.753158
Source_2,0.667700,0.541053
Target_0,0.805100,0.699474
Target_1,0.756900,0.626316
Target_2,0.597600,0.514211
mean,0.742267,0.624912
h_mean,0.732173,0.613979


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 600/600 [00:57<00:00, 10.37it/s]
2021-10-27 16:55:00,036 - pytorch_modeler.py - INFO - epoch:90/100, tr_loss:0.041226, src_loss:0.273237, src_mean_auc:0.765267, tgt_loss:0.258391, tgt_mean_auc:0.714500, mean_auc:0.739883,


,AUC,pAUC
Source_0,0.805700,0.663684
Source_1,0.843700,0.759474
Source_2,0.646400,0.531579
Target_0,0.819600,0.723158
Target_1,0.725400,0.594211
Target_2,0.598500,0.491053
mean,0.739883,0.627193
h_mean,0.727895,0.611820


 69%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋                                                             | 31/45 [00:33<00:14,  1.07s/it]

In [ ]:
#run(machine_types[0], dev_paths)

# run